<a href="https://colab.research.google.com/github/hannape/IBAC-Biodiv/blob/master/02_Test_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stworzenie macierzy testowej, niezbalansowanej. Dla 6 nagrań z 2016 powinna mieć pierwszy wymiar 30813, dla 14 nagreań z 2018: 71882

Kod wzięty z I2: 001_SplitRecordingAsFunction_Test_1reprV3_IBAC, wyczyszczony z niepotrzebnych rzeczy



In [0]:
## JAKO FUNKCJA

!pip install audiosegment
!pip install librosa
!pip install pydub
!pip install webrtcvad
!pip install librosa
!pip install -q spectrum
!pip install pkgconfig libtfr


In [0]:
import libtfr
import librosa
from scipy.io import wavfile
from random import choice
import librosa
from pydub import AudioSegment
from pydub.utils import make_chunks
import math
import audiosegment
from pydub import AudioSegment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
import glob
import os
import scipy
import librosa
from librosa import display
from librosa.core import power_to_db
from librosa.core import amplitude_to_db
import IPython.display as ipd
import time

In [0]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#### Parametry ####
             
plot_repr = 0          # czy wyrysowywać głosy wszystkie
play_sound = 0
minimal_duration = 4   # minimalny czas trwania głosu w ms, b zakwalifikowac chunka jako hasbird
f_min = 4000           # minimalna częstotliwość analizowana ze spektro
f_max = 9500           # maksymalna częstotliwość analizowana ze spektro
f_min2 = 4000           # minimalna częstotliwość analizowana ze spektro
f_max2 = 9500           # maksymalna częstotliwość analizowana ze spektro
n_mels = 60
n_mels2 = 80
step = 150

###################

def representations(frame):
  
      ####### 
      global result1,result1b,result3,result3b,result5,result5b
      
      fs = 44100
      samples = frame.get_array_of_samples()
      samples_float = librosa.util.buf_to_float(samples,n_bytes=2,dtype=np.float32)
      audio = np.array(samples_float)
      '''
      ####### reprezentacja 1  ------- 63 x 148 ------
      stft = librosa.stft(audio, n_fft=512, win_length=512, hop_length=150, window= "hann") #scipy.signal.get_window(('gaussian',50) ,512) )#scipy.signal.gaussian(512, 2)   ) 
      stft1 = amplitude_to_db(np.abs(stft)**2)
      freqs = librosa.core.fft_frequencies(n_fft=512, sr=44100)
      fmin =  min(freqs[(freqs >= f_min)])
      i,  = np.where( freqs >= min(freqs[(freqs >= f_min)]))
      j,  = np.where( freqs <= max(freqs[(freqs <= f_max)]))
      stft1 = stft1[min(i):max(j),]
      result1 = stft1
      
      ####### reprezentacja 1b - spektrogram większy hop length ------- 63 x 63 ------
      stft = librosa.stft(audio, n_fft=512, win_length=512, hop_length=355, window= "hann") #scipy.signal.get_window(('gaussian',50) ,512) )#scipy.signal.gaussian(512, 2)   ) 
      stft1 = amplitude_to_db(np.abs(stft)**2)
      freqs = librosa.core.fft_frequencies(n_fft=512, sr=44100)
      fmin =  min(freqs[(freqs >= f_min)])
      i,  = np.where( freqs >= min(freqs[(freqs >= f_min)]))
      j,  = np.where( freqs <= max(freqs[(freqs <= f_max)]))
      stft1 = stft1[min(i):max(j),]
      result1b = stft1
      
      ####### reprezentacja 3 - mel spektrogram ------- 60 x 111 ------
    
      # Compute Short-Term Fourier Transform (STFT).
      stft = librosa.stft(np.array(samples_float), n_fft=512, win_length=512, hop_length= 200,    window="hann")

      # Compute squared magnitude coefficients.
      abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
      result3 = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=44100, n_mels= 60, fmin = 4000, fmax=9500, hop_length=512, n_fft=512)
      result3 = 0.5 * librosa.amplitude_to_db(result3, ref=1.0)
      
      ####### reprezentacja 3b - mel spektrogram większy hop length ------- 60 x 63 ------
    
      # Compute Short-Term Fourier Transform (STFT).
      stft = librosa.stft(np.array(samples_float), n_fft=512, win_length=512, hop_length= 355,    window="hann")

      # Compute squared magnitude coefficients.
      abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
      result3 = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=44100, n_mels= 60, fmin = 4000, fmax=9500, hop_length=512, n_fft=512)
      result3b = 0.5 * librosa.amplitude_to_db(result3, ref=1.0)
      
      
      ####### reprezentacja 3 V3 - mel spektrogram  ------- 60 x 148 ------
    
      # Compute Short-Term Fourier Transform (STFT).
      stft = librosa.stft(np.array(samples_float), n_fft=512, win_length=512, hop_length= 150,    window="hann")

      # Compute squared magnitude coefficients.
      abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)
      result3 = librosa.feature.melspectrogram(y=None, S=abs2_stft, sr=44100, n_mels= 60, fmin = 4000, fmax=9500, hop_length=512, n_fft=512)
      result3 = 0.5 * librosa.amplitude_to_db(result3, ref=1.0)
      '''
      ####### reprezentacja 5 - multitaper ------- 64 x 61 ------
      
      result5 = libtfr.tfr_spec(audio, N = 512, step = 355, Np = 490, K = 2, tm = 1, flock =0.1, tlock = 10)     
      freqs, ind = libtfr.fgrid(fs, 512, fpass=(4000,9500)) 
      result5 = result5[ind,]; # tylko interesujące nas pasmo
      
      if(np.isinf(result5).any()):
        print("inf result B: " + str(i))
        
      '''      
      ####### reprezentacja 5b - multitaper o większej rozdzielczości ------- 64 x 149 ------
      
      result5b = libtfr.tfr_spec(audio, N = 512, step = 145, Np = 490, K = 2, tm = 1, flock =0.1, tlock = 10)     
      freqs, ind = libtfr.fgrid(fs, 512, fpass=(4000,9500)) 
      result5b = result5b[ind,]; # tylko interesujące nas pasmo
      
      if(np.isinf(result5b).any()):
        print("inf result B: " + str(i))  
      '''

In [0]:
def SplitRecordingFunctionTest(nazwa):

  name = nazwa + ".wav"
  name_txt = nazwa + ".txt" #dot.txt"  txt przecinkowe

  with open((source_labels + name_txt), 'r+') as f:  # zmiana na kropki
      text = f.read()
      f.seek(0)
      f.truncate()
      f.write(text.replace(',', '.'))
  fs, data = wavfile.read(source + name)   # wypluwa błąd, nie rozumie opisow dodanych przez Audacity pewnie
  print(name)
  print(source_labels + name_txt)
  myaudio = audiosegment.from_file(source + name)
  channel_count = myaudio.channels    #Get channels
  sample_width = myaudio.sample_width #Get sample width
  duration_in_ms = len(myaudio) #Length of audio in ms
  sample_rate = myaudio.frame_rate
  nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)
  print(nr_of_chunks)

  ########## Odczyt labeli z csv, wyczyszczenie zaznaczonych trzasków (t, ?)
  calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']
  if (os.stat(source_labels + name_txt).st_size != 0):
    y_read = pd.read_csv(source_labels + name_txt,  header=None, sep = "\t")
    y_read.columns = ["start", "end", "bird"]
    y_read_cleaned = y_read[~y_read['bird'].isin(calls_to_cut)]
    print(y_read_cleaned['bird'])
    y_read = y_read_cleaned
    not_empty = 1
  else:
    print(source_labels + name_txt)
    print("bez etykiet!!! : " + str(name))
    not_empty = 0
    sum_birds2 = 0
    y_read=[]
    birds_chunks = []
    df=pd.DataFrame()
  
  ########## Wyszukanie pozytywnych fragmentów - fragmentów zawierający głos, ilość takich ramek

  image_positive = []
  description_positive = []
  species=[]
  sum_birds=0
  has_bird = 1
  plot_repr = 0          # czy wyrysowywać głosy wszystkie
  play_sound = 1
  rep1 = []
  rep1b = []
  rep3 = []
  rep3b = []
  rep5 = []
  rep5b = []
  
  hasbird =[]
  spec= []
  labels = ['Chunk_nr', 'Chunk_start', 'Chunk_end', 'Call_start_chunk','Call_end_chunk','Call_start','Call_end','Has_bird','2nd_call_start','2nd_call_end']

  
  for i in range(0,math.floor(nr_of_chunks)):
    hadbird = 0;
    frame_start = i*(chunk_length_ms - chunk_overlap)
    frame_end = i*(chunk_length_ms - chunk_overlap)+ chunk_length_ms
    frame = myaudio[frame_start : frame_end]
    representations(frame)
    #rep1.append(result1)
    #rep1b.append(result1b)
    #rep3.append(result3)
    #rep3b.append(result3b)
    rep5.append(result5)
    #rep5b.append(result5b)
  
    x = range(frame_start,frame_end)

    if (len(y_read)!=0):
      for j in y_read.index.tolist():
          bird_start = int(round(y_read.start[j]*1000))
          bird_end = int(round(y_read.end[j]*1000))
          y = range(bird_start,bird_end)

          intersect_start = max(x[0], y[0])
          intersect_end = min(x[-1], y[-1])+1
          range1 = (range(intersect_start, intersect_end))

          if len(range1)> minimal_duration: 
            sum_birds = sum_birds+1
            hadbird=1
            representations(frame)  
            #image_positive.append(result1)  # nie musi być tutaj, dopiero po usunięciu duplikatów może?
            species.append(y_read.bird[j])
            hasbird.append([1])
            
            print(y_read.bird[j])

            #description_positive.append([i, frame_start, frame_end, intersect_start, intersect_end, bird_start, bird_end, has_bird, 0, 0])
          if (hadbird==1):
            break
       
    if (hadbird==0):
      species.append([0])
      hasbird.append([0])
      
    if (i%1000==0): # 500
      print("--- chunk nr %s recording %s---  " % (i, nazwa))

      
  #print(np.isinf(rep5b).any())   
  print(np.shape(rep3))  
  print(np.shape(species)) 
  print(np.shape(hasbird)) 
  
  # tylko 1 reprezentacja
  
  return rep5, hasbird, species
  
  #if not np.isinf(rep5b).any():
  #  np.savez((source_out + nazwa + name_out  ), rep1b = rep1b, hasbird=hasbird, species = species) #  freqs=freqs, times= times,


In [0]:
################################################################ MAIN

source = "drive/My Drive/nagrania/2016_stary_test/"
source_labels = "drive/My Drive/labele/2016_stary_test/"
dataset_nr = 1

#source = "drive/My Drive/nagrania/2018_nowy_test/"
#source_labels = "drive/My Drive/labele/2018_nowy_test/"
#dataset_nr = 2

source_out = "drive/My Drive/"
nazwa_list=[]

a = sorted(os.listdir(source_labels))
print(a)
for i in range(0,np.shape(a)[0]):
  nazwa_list.append(os.path.splitext(a[i])[0])
  print(os.path.splitext(a[i])[0])
  
#### Parametry ####
chunk_length_ms = 500
chunk_overlap = 150

###################  


['BUK4_20160922_005604.txt', 'BUK4_20161008_004605.txt', 'BUK4_20161016_012704.txt', 'BUK4_20161016_035704.txt', 'BUK5_20161101_002104a.txt', 'BUK5_20161101_002104b.txt']
BUK4_20160922_005604
BUK4_20161008_004605
BUK4_20161016_012704
BUK4_20161016_035704
BUK5_20161101_002104a
BUK5_20161101_002104b


In [0]:
repr_nr_out = "rep5_nowa676"

# pierwsze nagranie
rep1b, hasbird, species = SplitRecordingFunctionTest(nazwa_list[0])
print(np.shape(rep1b))
print(np.shape(hasbird))
print(np.shape(species))
X_test = rep1b
print(np.isinf(X_test).any())
y_test = hasbird
y_test_species =  species 
print(nazwa_list[0])
print("pozytytwnych ramek: ")
print(sum(np.asarray(y_test)))

for i in range(1,np.shape(nazwa_list)[0]):
  rep1b, hasbird, species = SplitRecordingFunctionTest(nazwa_list[i])
  X_test = np.concatenate((X_test,rep1b))
  y_test = np.concatenate((y_test,hasbird))
  #print(np.shape(y_test_species))
  print(np.isinf(X_test).any())
  a = species 
  
  if (len(np.shape(a))==2):  # jeśli dwa wymiary
     a = np.squeeze(species)  # to squeezujemy

  y_test_species = np.concatenate((y_test_species,a )) 

  print(np.shape(X_test))
  print(np.shape(y_test))
  print(np.shape(y_test_species))
  print(nazwa_list[i])
  print("pozytytwnych ramek: ")
  print(sum(np.asarray(y_test)))


print(np.shape(X_test))
print(np.shape(y_test))
print(np.shape(y_test_species)) 

print(np.isinf(X_test).any())


BUK4_20160922_005604.wav
drive/My Drive/labele/2016_stary_test/BUK4_20160922_005604.txt
5136.72
0       d
1       s
2       s
3       s
4       s
5       s
6       s
7       s
8       s
9       s
10      s
11      s
12      s
13      s
14      s
15      s
16     ni
17      s
18      s
19      s
20      s
21      s
22      s
23      s
24    ???
25      s
26      s
27     s?
28      s
29      s
30      s
31      s
32      s
Name: bird, dtype: object
--- chunk nr 0 recording BUK4_20160922_005604---  
d
d
s
s
s
s
s
--- chunk nr 1000 recording BUK4_20160922_005604---  
s
s
s
s
s
s
s
s
s
s
s
s
s
s
s
--- chunk nr 2000 recording BUK4_20160922_005604---  
s
s
s
s
ni
ni
--- chunk nr 3000 recording BUK4_20160922_005604---  
s
s
s
s
s
s
s
--- chunk nr 4000 recording BUK4_20160922_005604---  
s
s
s
s
s
s
???
???
s
s
s
s
s?
s?
s
s
s
s
s
s
s
s
--- chunk nr 5000 recording BUK4_20160922_005604---  
(0,)
(5136,)
(5136, 1)
(5136, 64, 61)
(5136, 1)
(5136,)
False
BUK4_20160922_005604
pozytytwnych ramek: 
[

In [0]:
print(np.shape(X_test))
print(sum(np.asarray(y_test)))

(30813, 64, 61)
[676]


In [0]:
np.save(('drive/My Drive/X_test' +  str(dataset_nr) + '_' + repr_nr_out), X_test)
np.save(('drive/My Drive/y_test' + str(dataset_nr) + '_' + repr_nr_out), y_test)
np.save(('drive/My Drive/y_species_test' + str(dataset_nr) + '_' + repr_nr_out), y_test_species)  
print('zapisane')

zapisane
